# HITS

In this notebook, we will use both NetworkX and cuGraph to compute HITS.  
The NetworkX and cuGraph processes will be interleaved so that each step can be compared.

Notebook Credits
* Original Authors: Bradley Rees and James Wyles
* Created:   06/09/2020
* Updated:   08/16/2020

RAPIDS Versions: 0.15  

Test Hardware

* GV100 32G, CUDA 10.0


## Introduction
HITS, also known as hubs and authorities, computes the relative importance of vertices.   

See [Wikipedia](https://en.wikipedia.org/wiki/HITS_algorithm) for more details on the algorithm.

HITS can be thought of as similar to PageRank, it is an iterative algorithm that propagates scores until a tolerance is reached, or max number of iterations is processed.  

---
From Wikepedia:

The algorithm performs a series of iterations, each consisting of two basic steps:

__Authority__ update: Update each node's authority score to be equal to the sum of the hub scores of each node that points to it. That is, a node is given a high authority score by being linked from pages that are recognized as Hubs for information.

__Hub__ update: Update each node's hub score to be equal to the sum of the authority scores of each node that it points to. That is, a node is given a high hub score by linking to nodes that are considered to be authorities on the subject.

---


To compute the HITS scores for a graph in cuGraph we use:<br>

**cugraph.hits(G,max_iter=100,
         tol=1.0e-5,
         nstart=None,
         normalized=True)**
* __G__: cugraph.Graph object
* __max_iter__: int, The maximum number of iterations before an answer is returned. 
* __tol__: float, Set the tolerance the approximation, this parameter should be a small magnitude value. 
* __nstart__: cudf.DataFrame - Not currently supported
* __normalized_ : bool - Not currently supported, always used as True

Returns:
* __df__: a cudf.DataFrame object with two columns:
  * df['vertex'] : cudf.Series
        Contains the vertex identifiers
  * df['hubs'] : cudf.Series
        Contains the hubs score
  * df['authorities'] : cudf.Series
        Contains the authorities score


### Some notes about vertex IDs...
* The current version of cuGraph requires that vertex IDs be representable as 32-bit integers, meaning graphs currently can contain at most 2^32 unique vertex IDs. However, this limitation is being actively addressed and a version of cuGraph that accommodates more than 2^32 vertices will be available in the near future.
* cuGraph will automatically renumber graphs to an internal format consisting of a contiguous series of integers starting from 0, and convert back to the original IDs when returning data to the caller. If the vertex IDs of the data are already a contiguous series of integers starting from 0, the auto-renumbering step can be skipped for faster graph creation times.
  * To skip auto-renumbering, set the `renumber` boolean arg to `False` when calling the appropriate graph creation API (eg. `G.from_cudf_edgelist(gdf_r, source='src', destination='dst', renumber=False)`).
  * For more advanced renumbering support, see the examples in `structure/renumber.ipynb` and `structure/renumber-2.ipynb`


### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)


### Prep

In [ ]:
# The notebook compares cuGraph to NetworkX,  
# therefore there some additional non-RAPIDS python libraries need to be installed. 
# Please run this cell if you need the additional libraries
!pip install networkx

In [ ]:
# Import needed libraries
import cugraph
import cudf

In [ ]:
# NetworkX libraries
import networkx as nx

### Some Prep

In [ ]:
# Define the path to the test data  
datafile='../data/karate-data.csv'

---
# NetworkX

In [ ]:
# Read the data, this also created a NetworkX Graph 
file = open(datafile, 'rb')
Gnx = nx.read_edgelist(file)

In [ ]:
hits_nx = nx.hits(Gnx)

In [ ]:
hits_nx

Running NetworkX is that easy.  
Let's seet how that compares to cuGraph

----

# cuGraph

### Read in the data - GPU
cuGraph depends on cuDF for data loading and the initial Dataframe creation

The data file contains an edge list, which represents the connection of a vertex to another.  The `source` to `destination` pairs is in what is known as Coordinate Format (COO).  In this test case, the data is just two columns.  However a third, `weight`, column is also possible

In [ ]:
# Read the data  
gdf = cudf.read_csv(datafile, names=["src", "dst"], delimiter='\t', dtype=["int32", "int32"] )

### Create a Graph 

In [ ]:
# create a Graph using the source (src) and destination (dst) vertex pairs from the Dataframe 
G = cugraph.Graph()
G.from_cudf_edgelist(gdf, source='src', destination='dst')

### Call the HITS algorithm

In [ ]:
# Call cugraph.hits to get the hits scores
gdf_hits = cugraph.hits(G)

_It was that easy!_  
Compared to NetworkX, the cuGraph data loading might have been more steps, but using cuDF allows for a wider range of data to be loaded. 


----

Let's now look at the results

In [ ]:
def print_hub_threshold(_df, t=0) :
    filtered = _df.query('hubs >= @t')
    
    for i in range(len(filtered)):
        print("Best vertex is " + str(filtered['vertex'].iloc[i]) + 
            " with HUB score of " + str(filtered['hubs'].iloc[i]))  

In [ ]:
def print_authorities_threshold(_df, t=0) :
    filtered = _df.query('authorities >= @t')
    
    for i in range(len(filtered)):
        print("Best vertex is " + str(filtered['vertex'].iloc[i]) + 
            " with Authorities score of " + str(filtered['authorities'].iloc[i]))  

In [ ]:
print_hub_threshold(gdf_hits, gdf_hits['hubs'].max())

In [ ]:
print_authorities_threshold(gdf_hits, gdf_hits['authorities'].max())

----

Since this is a very small graph, let's just sort and get the first three records

In [ ]:
sort_a = gdf_hits.sort_values('authorities', ascending=False)

In [ ]:
sort_a.head(3)

<img src="../img/zachary_graph_hits.png" width="600">

In [ ]:
sort_h = gdf_hits.sort_values('hubs', ascending=False)

In [ ]:
sort_h.head(32)

___
Copyright (c) 2019-2020, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___